# Lab 01: Data Collection and Preprocessing

This notebook demonstrates fundamental data collection and preprocessing techniques using Python and pandas.

---
## Step 1: Load the Dataset

In this step, we load the retail transactions CSV file using pandas and inspect the first few rows to understand the data structure.

In [ ]:
# Import pandas library
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('data/Retail_Transactions_Dataset.csv')

# Display the first 3 rows
df.head(3)

---
## Step 2: Data Structure Choice

### Why pandas DataFrame?

We use a **pandas DataFrame** to store our retail transactions data for the following reasons:

| Criteria | DataFrame Advantage |
|----------|---------------------|
| **Tabular format** | CSV data is naturally row-column structured; DataFrames mirror this exactly |
| **Mixed data types** | Our dataset contains strings (`Product`, `City`), numbers (`Total_Cost`, `Total_Items`), and dates (`Date`) — DataFrames handle heterogeneous columns efficiently |
| **Built-in methods** | Pandas provides optimized functions for filtering, grouping, aggregating, and cleaning data |
| **Memory efficiency** | DataFrames use NumPy arrays under the hood, enabling vectorized operations |
| **Ecosystem integration** | Seamless compatibility with visualization libraries (matplotlib, seaborn) and ML frameworks (scikit-learn) |

### Alternatives Considered

- **Python lists/dicts**: No built-in support for column operations or missing data handling
- **NumPy arrays**: Require homogeneous data types; not ideal for mixed-type tabular data
- **SQL database**: Overkill for a single-file analysis; adds setup complexity

**Conclusion**: pandas DataFrame is the optimal choice for exploratory data analysis and preprocessing of structured CSV data.

---
## Step 3: Transaction Class

We define a `Transaction` class to encapsulate individual transaction records. This provides:
- **Encapsulation**: Each transaction is a self-contained object
- **Data cleaning**: The `clean()` method standardizes and sanitizes field values
- **Calculations**: The `total()` method computes the transaction total

In [ ]:
class Transaction:
    """
    Represents a single retail transaction.
    
    Attributes:
        transaction_id: Unique identifier for the transaction
        date: Date of the transaction
        customer_name: Name of the customer
        product: Product purchased
        quantity: Number of items (Total_Items)
        price: Cost of the transaction (Total_Cost)
        city: City where the transaction occurred
        promotion: Promotion/coupon code applied
    """
    
    def __init__(self, row):
        """Initialize a Transaction from a DataFrame row."""
        self.transaction_id = row.get('Transaction_ID', None)
        self.date = row.get('Date', None)
        self.customer_name = row.get('Customer_Name', None)
        self.product = row.get('Product', None)
        self.quantity = row.get('Total_Items', 0)
        self.price = row.get('Total_Cost', 0.0)
        self.city = row.get('City', None)
        self.promotion = row.get('Promotion', None)
    
    def clean(self):
        """
        Clean and standardize transaction data.
        
        Returns:
            self: The Transaction instance (for method chaining)
        """
        # Strip whitespace from string fields
        if isinstance(self.customer_name, str):
            self.customer_name = self.customer_name.strip().title()
        
        if isinstance(self.product, str):
            self.product = self.product.strip().title()
        
        if isinstance(self.city, str):
            self.city = self.city.strip().title()
        
        if isinstance(self.promotion, str):
            self.promotion = self.promotion.strip().upper()
        
        # Handle missing/invalid numeric values
        try:
            self.quantity = int(self.quantity) if self.quantity else 0
        except (ValueError, TypeError):
            self.quantity = 0
        
        try:
            self.price = float(self.price) if self.price else 0.0
        except (ValueError, TypeError):
            self.price = 0.0
        
        return self
    
    def total(self):
        """
        Calculate the transaction total.
        
        Returns:
            float: The total cost of the transaction
        """
        return float(self.price)
    
    def __repr__(self):
        """String representation for debugging."""
        return f"Transaction(id={self.transaction_id}, product={self.product}, total={self.total():.2f})"


# --- Demo: Create and clean a Transaction ---
# Get the first row as a dictionary
sample_row = df.iloc[0].to_dict()

# Create a Transaction object
txn = Transaction(sample_row)

# Clean the transaction data
txn.clean()

# Display the transaction
print(f"Transaction ID: {txn.transaction_id}")
print(f"Customer:       {txn.customer_name}")
print(f"Product:        {txn.product}")
print(f"Quantity:       {txn.quantity}")
print(f"City:           {txn.city}")
print(f"Promotion:      {txn.promotion}")
print(f"Total:          ${txn.total():.2f}")

---
## Step 4: Bulk Load Transactions

Convert the entire DataFrame into a list of `Transaction` objects. This demonstrates:
- Iterating over DataFrame rows
- Creating objects in bulk
- Applying the `clean()` method to each transaction

In [ ]:
# Convert DataFrame rows to a list of Transaction objects
# Each row becomes a Transaction, then we call clean() on it

transactions = []

for index, row in df.iterrows():
    # Convert row to dictionary and create Transaction
    txn = Transaction(row.to_dict())
    
    # Clean the transaction data
    txn.clean()
    
    # Add to list
    transactions.append(txn)

# Display summary
print(f"Total transactions loaded: {len(transactions):,}")
print(f"\nFirst 3 transactions:")
for t in transactions[:3]:
    print(f"  {t}")

print(f"\nLast 3 transactions:")
for t in transactions[-3:]:
    print(f"  {t}")

---
## Step 5: Data Profiling

Compute basic statistics to understand the dataset:
- **Price statistics**: min, mean, max
- **Unique cities**: count of distinct shipping locations

In [ ]:
# Extract prices from all transactions
prices = [t.price for t in transactions]

# Compute price statistics
min_price = min(prices)
max_price = max(prices)
mean_price = sum(prices) / len(prices)

# Get unique cities
cities = set(t.city for t in transactions)
unique_city_count = len(cities)

# Display profiling results
print("=" * 40)
print("DATA PROFILING RESULTS")
print("=" * 40)

print(f"\n📊 Price Statistics:")
print(f"   Min:  ${min_price:,.2f}")
print(f"   Mean: ${mean_price:,.2f}")
print(f"   Max:  ${max_price:,.2f}")

print(f"\n🏙️ City Statistics:")
print(f"   Unique cities: {unique_city_count}")

print(f"\n📋 Sample cities:")
for city in sorted(cities)[:5]:
    print(f"   - {city}")